In [1]:
import os
end_date="2017-12-27"
if not os.path.exists('/home/jian/Projects/LIFT/DBM/Output/'+end_date):
    os.makedirs('/home/jian/Projects/LIFT/DBM/Output/'+end_date)

In [2]:
import glob
import pandas as pd
import numpy as np
folder="/home/jian/Projects/LIFT/DBM/Data from API/"+end_date+"/"
file_list=[]
for file_name in glob.glob(folder+'*.csv'):
    df = pd.read_csv(file_name,encoding="ISO-8859-1",error_bad_lines=False)
    inds = min(df['Line Item'].index[df['Line Item'].apply(lambda x: pd.isnull(x))])
    df = df.iloc[0:inds,:]
    file_list.append(df)



b'Skipping line 3391: expected 24 fields, saw 27\n'
b'Skipping line 718: expected 24 fields, saw 27\n'
b'Skipping line 1875: expected 24 fields, saw 27\n'
b'Skipping line 175702: expected 25 fields, saw 27\n'
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 5643: expected 25 fields, saw 27\n'
b'Skipping line 2532: expected 25 fields, saw 27\n'
b'Skipping line 136368: expected 26 fields, saw 27\n'
b'Skipping line 807: expected 24 fields, saw 27\n'
b'Skipping line 462: expected 25 fields, saw 27\n'
b'Skipping line 308: expected 24 fields, saw 27\n'
b'Skipping line 3194: expected 24 fields, saw 27\n'
b'Skipping line 1878: expected 24 fields, saw 27\n'


In [3]:
Columns=["App/URL","Browser","ISP or Carrier","Channel","Channel ID","City","City ID","Creative","Creative Width","Creative Height","Creative Type","Date",
          "Day of Week","Device Type","DMA Name","Operating System","Region","Time of Day"]
HVA=["Snap Fitness - Franchise Opportunities",
      "Snap Fitness - Franchise FAQ",
      "Snap Fitness - Franchise - What is This? Link",
      "Snap Fitness - Franchise Success Stories",
      "Snap Fitness - Franchise Request Info",
      "Snap Fitness - Franchise Thank You"]
matrix=["Clicks","Impressions","Cost","HVA","CPHVA","CPC","CTR","CPM"]
Juba_Line_Items=["Juba BT - Health & Fitness - Geo 1",
                  "Juba BT - Sports - Geo 2 (DMAs)",
                  "Juba BT - Health & Fitness - Geo 2 (DMAs)",
                  "Juba BT - Gamers - Geo 1",
                  "Juba BT - Employment - Geo 1",
                  "Juba BT - Technophiles - Geo 1",
                  "Juba BT - Movie Lover - Geo 1",
                  "Juba BT - Sports - Geo 1",
                  "Juba BT - Travel - Geo 1",
                  "Juba BT - Gamers - Geo 2 (DMAs)",
                  "Juba BT - News Junkies - Geo 2 (DMAs)",
                  "Juba BT - Movie Lover - Geo 2 (DMAs)",
                  "Juba BT - Travel - Geo 2 (DMAs)",
                  "Juba BT - Technophiles - Geo 2 (DMAs)",
                  "Juba BT - Finance - Geo 2 (DMAs)",
                  "Juba BT - Auto - Geo 2 (DMAs)",
                  "Juba BT - Employment - Geo 2 (DMAs)",
                  "Juba BT - New Mover - Geo 2 (DMAs)",
                  "Juba BT - Finance - Geo 1",
                  "Juba BT - New Mover - Geo 1",
                  "Juba BT - Auto - Geo 1",
                  "Juba BT - News Junkies - Geo 1"]
Keyword_Line_Items=["Keyword CT DESK","Keyword CT MOB"]
Retarget_Line_Items=["Site RT DESK"]
LAL_Line_Items=["LAL"]

In [11]:
len(Juba_Line_Items)+len(Keyword_Line_Items)+len(Retarget_Line_Items)+len(LAL_Line_Items)

26

In [4]:
writer_list=[[],[],[],[],[],[],[],[],[],[],[],[],[]]
for i in range(len(file_list)):
    df=file_list[i]
    df_JUBA=df[df['Line Item'].isin(Juba_Line_Items)]
    df_Keyword=df[df['Line Item'].isin(Keyword_Line_Items)]
    df_Retarget=df[df['Line Item'].isin(Retarget_Line_Items)]
    df_LAL=df[df['Line Item'].isin(LAL_Line_Items)]

    Four_Type_List=[df_JUBA,df_Keyword,df_Retarget,df_LAL] #Actually 4 type, one new "LAL" added (not yet)

    
    for j in range(len(Four_Type_List)):

        df_report_type=Four_Type_List[j]
        df_non_hva=df_report_type[[col for col in df_report_type.columns if col in Columns]+["Impressions","Clicks","Revenue (Adv Currency)","Total Conversions"]]
        
        if len(df_non_hva.columns)==5:
            df_merge=pd.DataFrame
            df_hva=df_hva=df_report_type[df_report_type["Conversion Pixel"] !="Total"][[col for col in df_report_type.columns if col in Columns]+["Conversion Pixel","Total Conversions"]]
            df_hva=df_hva.groupby([col for col in df_report_type.columns if col in Columns]+['Conversion Pixel'])['Total Conversions'].sum().to_frame()
            df_hva.reset_index(inplace=True)
            df_hva=df_hva.pivot(index=[col for col in df_report_type.columns if col in Columns][0],columns="Conversion Pixel",values="Total Conversions")
            df_hva.reset_index(inplace=True)
            df_hva=df_hva.fillna(0)
            
            df_non_hva=df_report_type[df_report_type["Conversion Pixel"] =="Total"][[col for col in df_report_type.columns if col in Columns]+["Impressions","Clicks","Revenue (Adv Currency)"]]
            df_non_hva=df_non_hva.groupby([col for col in df_report_type.columns if col in Columns])["Impressions","Clicks","Revenue (Adv Currency)"].sum()
            df_non_hva.reset_index(inplace=True)

            df_merge=pd.merge(df_non_hva,df_hva,on=[col for col in df_report_type.columns if col in Columns],how="outer")
        
        
        if len(df_non_hva.columns)==6:
            df_merge=pd.DataFrame
            df_hva=df_hva=df_report_type[df_report_type["Conversion Pixel"] !="Total"][[col for col in df_report_type.columns if col in Columns]+["Conversion Pixel","Total Conversions"]]
            df_hva=df_hva.groupby([col for col in df_report_type.columns if col in Columns]+['Conversion Pixel'])['Total Conversions'].sum().to_frame()
            df_hva.reset_index(inplace=True)
            df_hva=df_hva.pivot_table(index=[str([col for col in df_report_type.columns if col in Columns][0]),str([col for col in df_report_type.columns if col in Columns][1])],columns="Conversion Pixel",values="Total Conversions")
            df_hva.reset_index(inplace=True)
            df_hva=df_hva.fillna(0)
            
            df_non_hva=df_report_type[df_report_type["Conversion Pixel"] =="Total"][[col for col in df_report_type.columns if col in Columns]+["Impressions","Clicks","Revenue (Adv Currency)"]]
            df_non_hva=df_non_hva.groupby([col for col in df_report_type.columns if col in Columns])["Impressions","Clicks","Revenue (Adv Currency)"].sum()
            df_non_hva.reset_index(inplace=True)

            df_merge=pd.merge(df_non_hva,df_hva,on=[col for col in df_report_type.columns if col in Columns],how="outer")

        # df_merge=df_merge.fillna(0)

        if 'Snap Fitness - Franchise Thank You' not in df_merge.columns.values.tolist():
            df_merge.rename(columns={'Revenue (Adv Currency)':'Cost'}, inplace=True)
            df_merge['HVA']=df_merge[[col for col in df_merge.columns if col in HVA]].sum(axis=1)
            df_merge['CPHVA']=df_merge['Cost']/df_merge['HVA']
            df_merge['CPC']=df_merge['Cost']/df_merge['Clicks']
            df_merge['CTR']=df_merge['Clicks']/df_merge['Impressions']
            df_merge['CPM']=df_merge['Cost']/df_merge['Impressions']*1000
            df_merge.reset_index(inplace=True)
            df_merge=df_merge[[col for col in df_report_type.columns if col in Columns]+["Impressions","Clicks","Cost","HVA","CTR","CPHVA","CPC","CPM"]+[col for col in df_merge.columns if col in HVA]]
            if str([col for col in df_report_type.columns if col in Columns][0]) != "Date":
                df_merge=df_merge.sort_values(['CPHVA','CTR','CPC','CPM'], ascending=[True,False,True,True])
        
            writer_list[i].append(df_merge)
              

 
        if 'Snap Fitness - Franchise Thank You' in df_merge.columns.values.tolist():

            df_merge.rename(columns={'Revenue (Adv Currency)':'Cost'}, inplace=True)
            df_merge['CP_ThankYou_Page']=df_merge['Cost']/df_merge['Snap Fitness - Franchise Thank You']
            df_merge['HVA']=df_merge[[col for col in df_merge.columns if col in HVA]].sum(axis=1)
            df_merge['CPHVA']=df_merge['Cost']/df_merge['HVA']
            df_merge['CPC']=df_merge['Cost']/df_merge['Clicks']
            df_merge['CTR']=df_merge['Clicks']/df_merge['Impressions']
            df_merge['CPM']=df_merge['Cost']/df_merge['Impressions']*1000
            df_merge.reset_index(inplace=True)
            df_merge=df_merge[[col for col in df_report_type.columns if col in Columns]+["Impressions","Clicks","Cost","HVA","CTR","CP_ThankYou_Page","CPHVA","CPC","CPM"]+[col for col in df_merge.columns if col in HVA]]

            if str([col for col in df_report_type.columns if col in Columns][0]) != "Date":
                df_merge=df_merge.sort_values(['CP_ThankYou_Page','CPHVA','CTR','CPC','CPM'], ascending=[True,True,False,True,True])

            writer_list[i].append(df_merge)

        
        # df_merge=df_merge[,c(names(df_merge)[which(names(df_merge) %ni% matrix & names(df_merge) %ni% Total_HVA_Reshaped)],matrix, names(df_merge)[which(names(df_merge) %in% Total_HVA_Reshaped)])]
        # names(df_merge)=c(names(df_merge)[1:which(names(df_merge)=="CPM")],sapply(names(df_merge)[(which(names(df_merge)=="CPM")+1):ncol(df_merge)],function(x) substr(x,(nchar("Total.Conversions.")+1),nchar(x))))
        # df_merge=df_merge[order(df_merge$CPHVA,df_merge$CTR,df_merge$CPC,df_merge$CPM,decreasing = c(FALSE,TRUE,FALSE,FALSE)),]
        # df_merge$Thank.You.Page<-rep(NA,nrow(df_merge))
        # writer[[j]][[i]]=df_merge
    

In [5]:
writer_JUBA=pd.ExcelWriter("/home/jian/Projects/LIFT/DBM/Output/"+end_date+"/"+end_date+'_JUBA.xlsx', engine='xlsxwriter')
writer_Kw=pd.ExcelWriter("/home/jian/Projects/LIFT/DBM/Output/"+end_date+"/"+end_date+'_Kw.xlsx', engine='xlsxwriter')
writer_Retarget=pd.ExcelWriter("/home/jian/Projects/LIFT/DBM/Output/"+end_date+"/"+end_date+'_Retarget.xlsx', engine='xlsxwriter')
writer_LAL=pd.ExcelWriter("/home/jian/Projects/LIFT/DBM/Output/"+end_date+"/"+end_date+'_LAL.xlsx', engine='xlsxwriter')


for i in range(len(file_list)):
    writer_list[i][0].to_excel(writer_JUBA,str(writer_list[i][0].columns[0]).replace("/"," "),index=False)
    writer_list[i][1].to_excel(writer_Kw,str(writer_list[i][0].columns[0]).replace("/"," "),index=False)
    writer_list[i][2].to_excel(writer_Retarget,str(writer_list[i][0].columns[0]).replace("/"," "),index=False)
    writer_list[i][3].to_excel(writer_LAL,str(writer_list[i][0].columns[0]).replace("/"," "),index=False)

In [6]:
workbook_JUBA  = writer_JUBA.book
format1_Dollar = workbook_JUBA.add_format({'num_format': '[$$-409]#,##0.00'})
format2_Pctg = workbook_JUBA.add_format({'num_format': '0.00%'})

Date_JUBA=writer_JUBA.sheets['Date']
Day_of_Week_JUBA=writer_JUBA.sheets['Day of Week']
Time_of_Day_JUBA=writer_JUBA.sheets['Time of Day']
City_JUBA=writer_JUBA.sheets['City'] #2
DMA_Name_JUBA=writer_JUBA.sheets['DMA Name']
Region_JUBA=writer_JUBA.sheets['Region']
App_URL_JUBA=writer_JUBA.sheets['App URL']
Browser_JUBA=writer_JUBA.sheets['Browser']
Creative_JUBA=writer_JUBA.sheets['Creative']
Creative_Width_JUBA=writer_JUBA.sheets['Creative Width'] #2
Device_Type_JUBA=writer_JUBA.sheets['Device Type']
ISP_or_Carrier_JUBA=writer_JUBA.sheets['ISP or Carrier']
Operating_System_JUBA=writer_JUBA.sheets['Operating System']

One_Index_List=[Date_JUBA,Day_of_Week_JUBA,Time_of_Day_JUBA,DMA_Name_JUBA,Region_JUBA,App_URL_JUBA,Browser_JUBA,Creative_JUBA,
               Device_Type_JUBA,ISP_or_Carrier_JUBA,Operating_System_JUBA]
Two_Index_List=[City_JUBA,Creative_Width_JUBA]

for one_index_worksheet in One_Index_List:
    one_index_worksheet.set_column('D:D',None,format1_Dollar)
    one_index_worksheet.set_column('G:J',None,format1_Dollar)
    one_index_worksheet.set_column('F:F',None,format2_Pctg)
    
for two_index_worksheet in Two_Index_List:
    two_index_worksheet.set_column('E:E',None,format1_Dollar)
    two_index_worksheet.set_column('H:K',None,format1_Dollar)
    two_index_worksheet.set_column('G:G',None,format2_Pctg)

Creative_Width_JUBA.Name = 'Creative Size'


In [7]:
workbook_Kw  = writer_Kw.book
format1_Dollar = workbook_Kw.add_format({'num_format': '[$$-409]#,##0.00'})
format2_Pctg = workbook_Kw.add_format({'num_format': '0.00%'})

Date_Kw=writer_Kw.sheets['Date']
Day_of_Week_Kw=writer_Kw.sheets['Day of Week']
Time_of_Day_Kw=writer_Kw.sheets['Time of Day']
City_Kw=writer_Kw.sheets['City'] #2
DMA_Name_Kw=writer_Kw.sheets['DMA Name']
Region_Kw=writer_Kw.sheets['Region']
App_URL_Kw=writer_Kw.sheets['App URL']
Browser_Kw=writer_Kw.sheets['Browser']
Creative_Kw=writer_Kw.sheets['Creative']
Creative_Width_Kw=writer_Kw.sheets['Creative Width'] #2
Device_Type_Kw=writer_Kw.sheets['Device Type']
ISP_or_Carrier_Kw=writer_Kw.sheets['ISP or Carrier']
Operating_System_Kw=writer_Kw.sheets['Operating System']

One_Index_List=[Date_Kw,Day_of_Week_Kw,Time_of_Day_Kw,DMA_Name_Kw,Region_Kw,App_URL_Kw,Browser_Kw,Creative_Kw,
               Device_Type_Kw,ISP_or_Carrier_Kw,Operating_System_Kw]
Two_Index_List=[City_Kw,Creative_Width_Kw]

for one_index_worksheet in One_Index_List:
    one_index_worksheet.set_column('D:D',None,format1_Dollar)
    one_index_worksheet.set_column('G:J',None,format1_Dollar)
    one_index_worksheet.set_column('F:F',None,format2_Pctg)
    
for two_index_worksheet in Two_Index_List:
    two_index_worksheet.set_column('E:E',None,format1_Dollar)
    two_index_worksheet.set_column('H:K',None,format1_Dollar)
    two_index_worksheet.set_column('G:G',None,format2_Pctg)

Creative_Width_Kw.Name = 'Creative Size'


In [8]:
workbook_Retarget  = writer_Retarget.book
format1_Dollar = workbook_Retarget.add_format({'num_format': '[$$-409]#,##0.00'})
format2_Pctg = workbook_Retarget.add_format({'num_format': '0.00%'})

Date_Retarget=writer_Retarget.sheets['Date']
Day_of_Week_Retarget=writer_Retarget.sheets['Day of Week']
Time_of_Day_Retarget=writer_Retarget.sheets['Time of Day']
City_Retarget=writer_Retarget.sheets['City'] #2
DMA_Name_Retarget=writer_Retarget.sheets['DMA Name']
Region_Retarget=writer_Retarget.sheets['Region']
App_URL_Retarget=writer_Retarget.sheets['App URL']
Browser_Retarget=writer_Retarget.sheets['Browser']
Creative_Retarget=writer_Retarget.sheets['Creative']
Creative_Width_Retarget=writer_Retarget.sheets['Creative Width'] #2
Device_Type_Retarget=writer_Retarget.sheets['Device Type']
ISP_or_Carrier_Retarget=writer_Retarget.sheets['ISP or Carrier']
Operating_System_Retarget=writer_Retarget.sheets['Operating System']

One_Index_List=[Date_Retarget,Day_of_Week_Retarget,Time_of_Day_Retarget,DMA_Name_Retarget,Region_Retarget,App_URL_Retarget,Browser_Retarget,Creative_Retarget,
               Device_Type_Retarget,ISP_or_Carrier_Retarget,Operating_System_Retarget]
Two_Index_List=[City_Retarget,Creative_Width_Retarget]

for one_index_worksheet in One_Index_List:
    one_index_worksheet.set_column('D:D',None,format1_Dollar)
    one_index_worksheet.set_column('G:J',None,format1_Dollar)
    one_index_worksheet.set_column('F:F',None,format2_Pctg)
    
for two_index_worksheet in Two_Index_List:
    two_index_worksheet.set_column('E:E',None,format1_Dollar)
    two_index_worksheet.set_column('H:K',None,format1_Dollar)
    two_index_worksheet.set_column('G:G',None,format2_Pctg)

Creative_Width_Retarget.Name = 'Creative Size'


In [9]:
workbook_LAL  = writer_LAL.book
format1_Dollar = workbook_LAL.add_format({'num_format': '[$$-409]#,##0.00'})
format2_Pctg = workbook_LAL.add_format({'num_format': '0.00%'})

Date_LAL=writer_LAL.sheets['Date']
Day_of_Week_LAL=writer_LAL.sheets['Day of Week']
Time_of_Day_LAL=writer_LAL.sheets['Time of Day']
City_LAL=writer_LAL.sheets['City'] #2
DMA_Name_LAL=writer_LAL.sheets['DMA Name']
Region_LAL=writer_LAL.sheets['Region']
App_URL_LAL=writer_LAL.sheets['App URL']
Browser_LAL=writer_LAL.sheets['Browser']
Creative_LAL=writer_LAL.sheets['Creative']
Creative_Width_LAL=writer_LAL.sheets['Creative Width'] #2
Device_Type_LAL=writer_LAL.sheets['Device Type']
ISP_or_Carrier_LAL=writer_LAL.sheets['ISP or Carrier']
Operating_System_LAL=writer_LAL.sheets['Operating System']

One_Index_List=[Date_LAL,Day_of_Week_LAL,Time_of_Day_LAL,DMA_Name_LAL,Region_LAL,App_URL_LAL,Browser_LAL,Creative_LAL,
               Device_Type_LAL,ISP_or_Carrier_LAL,Operating_System_LAL]
Two_Index_List=[City_LAL,Creative_Width_LAL]

for one_index_worksheet in One_Index_List:
    one_index_worksheet.set_column('D:D',None,format1_Dollar)
    one_index_worksheet.set_column('G:J',None,format1_Dollar)
    one_index_worksheet.set_column('F:F',None,format2_Pctg)
    
for two_index_worksheet in Two_Index_List:
    two_index_worksheet.set_column('E:E',None,format1_Dollar)
    two_index_worksheet.set_column('H:K',None,format1_Dollar)
    two_index_worksheet.set_column('G:G',None,format2_Pctg)

Creative_Width_LAL.Name = 'Creative Size'


In [10]:
writer_JUBA.save()
writer_Kw.save()
writer_Retarget.save()
writer_LAL.save()